In [1]:
import tensorflow as tf
import keras
from keras import backend as K
from tensorflow.python.client import device_lib
from statsmodels.base.model import GenericLikelihoodModel,\
        GenericLikelihoodModelResults
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import pandas as pd
import re
import numpy as np
from tensorflow.keras.constraints import max_norm
from sklearn.model_selection import train_test_split
import requests
from nltk import tokenize
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU, LSTM
from keras.layers import Embedding
from timeit import default_timer as timer
import math
import random
from collections import defaultdict
from keras.models import model_from_json
import h5py
import numpy as np
import sentencepiece as spm
from sklearn.model_selection import train_test_split
import collections
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras import backend as K
import scipy
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from keras.callbacks import EarlyStopping
from sklearn.metrics import log_loss, make_scorer
import time
from sklearn.model_selection import StratifiedKFold
from keras.regularizers import l1
from keras.regularizers import l2
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
import codecs
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
from statsmodels.nonparametric.smoothers_lowess import lowess
from scipy.special import zeta
from scipy.stats import binom
from scipy.special import zeta
from scipy.misc import derivative
from scipy.stats import mannwhitneyu
from collections import Counter
print(tf.__version__)

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')

print('Found GPU at: {}'.format(device_name))
lg = np.log10

2.4.1
Found GPU at: /device:GPU:0


In [3]:
text = '1000000_26_2'

file = 'CLEAN_' + text + '.txt'
path = text + '/'


direc = text + '/'
os.mkdir(text)

# Cleans the file of weird characters etc. Then creates a new one in a seperate directory, this is mainly all
# done for mapping purposes, so i have everything in the right place.
infile = codecs.open('texts/' + text + '.txt','r',encoding='utf-8',errors='ignore')
outfile = codecs.open(direc + 'CLEAN_' + text + '.txt','w',encoding='utf-8',errors='ignore')

for sent in infile.readlines():
    a = ''.join([i for i in sent if not re.findall("[^\u0000-\u05C0\u2100-\u214F]+",i)])
    if len(a.split()) > 2:
        outfile.write(a)
outfile.close()

with open(path + file, encoding="utf8") as handle:
    sentences =  [l.strip()[0:-1] for l in handle.readlines()]

sentences1 = []
for sent in sentences:
    sent = re.sub(r"\s+", " ", sent)
    sentences1.append(sent.strip())


# Remove sentences shorter than 50 and remove 1/3 of the data
x = []
for i in sentences1:
    if len(i.split()) > 2 and len(i.split()) < 50:
        x.append(i)
x = x[0:(int(len(x)/1.5))]

# Calculate vocab size and make V a fraction of that
data = ' '.join(x)
word_tokeniser = Tokenizer()
word_tokeniser.fit_on_texts([data])
v = len(word_tokeniser.word_index) + 1
v = int(v*0.06)
print(v)

# Train sentence piece model and save it in the directory of the clean text
spm.SentencePieceTrainer.train(input=direc + 'CLEAN_' + text + '.txt', model_prefix=direc + 'encoding' , vocab_size=v)


# Create a txtfile with info about the text, typicality etc.
txtfile = codecs.open(direc + 'textinfo.txt','w',encoding='utf-8',errors='ignore')
txtfile.write("Vocab size = " + str(v) + '\n')
txtfile.write("Amount of sentences = " + str(len(x))+ '\n')


sp = spm.SentencePieceProcessor(model_file=path + 'encoding.model')
sp.SetEncodeExtraOptions('bos:eos')


10036
MANDELBROT
  Optimal Parameters (1.14, 3.87)
  Standard Error [relative]: (0.00035, 0.0244), [(0.000307, 0.00631)]
  Pseudo R^2: 0.657
  BIC [relative]: 3640000.0, [2.91]


True

In [4]:

X = []
X1 = []
y = []

# Encode sentences with 
for sentence in x:
    count = 1
    temp = sp.encode(sentence)
    for subword in temp:
        X.append(temp[0:count])
        if count < len(temp):
            y.append(temp[count])
        count+=1

X1 = []
for A in X:
    if 2 not in A:
        X1.append(A)

X = X1
print(X[0])
print(y[0])

print(X[1])
print(y[1])
y = to_categorical(y, num_classes=v)

[1]
2109
[1, 2109]
6422


In [5]:
# Max sent length for padding
max1 = 0
for i in X:
    if len(i) > max1:
        max1 = len(i)
print(max1)
print(len(X))
print(len(y))

167
1217325
1217325


In [6]:
X1 = []
for XD in X:
    X1.append(pad_sequences([XD], maxlen=max1, padding='pre'))
    
X1 = np.asarray(X1)

X2 = []
for i in X1:
    X2.append(i[0])
    
    
X2 = np.asarray(X2)
X = X2
X2 = []
X1 = []
np.shape(X)

(1217325, 167)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [ ]:
X_train[0:5]

In [ ]:
y_train[0:5]

In [ ]:
np.set_printoptions(precision=None, suppress=True)
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2, restore_best_weights=True)

In [ ]:
# Perplexity has to be calculated in batches or it crashes.
def perplexity(Xp, yp, model):
    chance = 0
    current_index = 0
    batch_size = 500
    rang = int(len(Xp) / batch_size) - 1
    length = batch_size * rang
    for _ in range(rang):
        distributionList = model.predict(Xp[current_index:current_index + batch_size], batch_size)
    
        tempcount = current_index
        for distribution in distributionList:
            chance += math.log2(distribution[np.nonzero(yp[tempcount])[0][0]])
            tempcount += 1
            
        current_index = current_index + batch_size 
    return 2**((-1/length)* chance)

In [ ]:
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath= "modelcheckP.h5",
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)


optimizer = keras.optimizers.Adam(0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
# Create the model

model = Sequential()

# V = vocab size
model.add(Embedding(v, 200, input_length = max1))
model.add(CuDNNLSTM(256 return_sequences=True))

model.add(Dropout(0.2))
model.add(CuDNNLSTM(256))

model.add(Dense(v, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=optimizer)


history = model.fit(X_train, y_train, epochs=100, verbose=1, batch_size=256, callbacks=[early_stopping,model_checkpoint_callback], validation_data=(X_test, y_test))



perplex = perplexity(X_test, y_test, model)
print(perplex)
   
